# Test Coverage

Writing tests for your application lets you check that the code you wrote works the way you expect. `pytest-cov` is a mature full-featured framework to write your tests efficiently and producting coverage reports. It removes all the overheads for creating tests.

You should test as much of your code as possible. The closer you get to 100% coverage, the more comfortable you can be that making a change won’t unexpectedly change other behavior. However, 100% coverage doesn’t guarantee that your application doesn’t have bugs. Despite this, code and test coverage is an important tool to use during development.

Code coverage is a measurement of how many lines/blocks of your code are executed while the automated tests are running. With testing, code coverage indicates how well your test set is covering your source code (i.e. to what extent is the source code covered by the set of test cases). Having "100% code coverage" doesn't mean everything is tested completely. While it means every line of code is tested, it doesn't mean they are tested under every situation.

### Install pytest-cov

The `azure-pipelines.yml` contains the below snippet to install `pytest-cov` for generating tests and coverage reports.

````
- script: |
    pip install pytest-cov
  displayName: 'Install Test and Coverage Libraries'
````

### Run Tests and Coverage

The below task in `azure-pipelines.yml` is the main section that runs tests and generates reports. The `inlineScript` points to the `testing` folder and runs tests located in it. The schema file is passed as an argument and the test reports are generated in `junit-xml` format.

````
- task: AzureCLI@1
  displayName: 'Run Tests and Coverage'
  inputs:
    azureSubscription: 'serviceConnection'
    scriptLocation: inlineScript
    inlineScript: 'py.test devops/code/testing/ --name devops/data_sample/predmain_good_schema.csv --junitxml=reports/test_report.xml'
````

### Test Code

The `devops/code/testing` folder contains the different tests that are run. The following are the different types of tests performed:

| Name   |      Description      |
|----------|-------------|
| _test_registered_model_ |  Tests if model.pkl is registered |
| _test_registered_model_tags_ |    Confirms if meaningful tags are used for the experiment   |
| _test_scoring_image_present_ | Checks if an image is deployed with model.pkl |
| _test_registered_model_metric_ | Checks if a new model is promoted and if it's accuracy is above 85% |
| _test_schema_types_ | Checks if the data contains float values |
| _test_schema_cols_ | Checks schema for the right number of columns |

### Publish Test Results

This task publishes test results to Azure Pipelines when tests are executed to provide a comprehensive test reporting and analytics experience. 

On each step and job, you can specify the conditions under which the step or job will run. With this step, we will publish test results irrespective of whether a previous task has succeeded or failed. The test report generated using pytest-cov is passed in this task.

````
- task: PublishTestResults@2
  inputs:
    testResultsFiles: 'reports/test_report.xml'
    testRunTitle: 'ADPM Tests: $(Agent.OS) - $(Build.DefinitionName)'
  condition: succeededOrFailed()
````

The published test results are displayed in the `Tests` tab in the pipeline summary and help you to measure pipeline quality and troubleshoot issues.

![publishedTests](../images/publishedTests.png)

You will also be able to view the published test results in the `Summary` tab of Azure pipelines portal along with build and deployments information.

### Exercise

1. The specific tests run can also be filtered by test name or outcome in the `Tests` tab. Can you change the filters to list all the tests that were run?

2. The log from `Run Tests and Coverage` will indicate code coverage as shown below. Can you explain why only 66% is covered with `test_artifacts.py`?

````
============================= test session starts ==============================
platform linux -- Python 3.6.8, pytest-4.3.1, py-1.8.0, pluggy-0.9.0
rootdir: /home/vsts/work/1/s, inifile:
plugins: cov-2.6.1
collected 6 items

devops/code/testing/test_artifacts.py ....                               [ 66%]
devops/code/testing/test_data.py ..                                      [100%]
````


3. Can you create another test in `devops/code/testing/test_data.py` to check if the values of the data sample are within a range (min/max or sd)? Perform another build and check to see if the `Tests` tab of the portal got updated.

    ````python
        === Solution ===
        def test_schema_types(get_sample):
            min = -10.0
            max = 10.0
            for col in data:
                try:
                    val = float(col)
                    assert min <= val <= max
                except ValueError:
                    raise AssertionError
    ````

4. The pipeline also performs ACI Service Test using the below task. However, this test is not published. Can you integrate this test with `pytest-cov` and publish the result?

````
- task: AzureCLI@1
  displayName: 'ACI Service Test'
  inputs:
    azureSubscription: 'serviceConnection'
    scriptLocation: inlineScript
    inlineScript: 'python devops/code/aci_service_test.py'
````